In [1]:
import pandas as pd
import numpy as np

# Paso 1: Definición de la Red
nodes = [1, 2, 3, 4, 5, 6]
links = {
    (1, 2): {'distance': 1},
    (2, 1): {'distance': 1},
    (1, 4): {'distance': 1},
    (4, 1): {'distance': 1},
    (2, 3): {'distance': 1},
    (3, 2): {'distance': 1},
    (2, 5): {'distance': 1},
    (5, 2): {'distance': 1},
    (3, 6): {'distance': 1},
    (6, 3): {'distance': 1},
    (4, 5): {'distance': 1},
    (5, 4): {'distance': 1},
    (5, 6): {'distance': 1},
    (6, 5): {'distance': 1},
}

# Paso 2: Recolección de Datos
modes = ['Auto', 'Bus', 'Camion']
movements = ['Izquierda', 'Recto', 'Derecha']

traffic_counts = {
    1: {
        'Entradas': {
            'Auto': {'Izquierda': 50, 'Recto': 200, 'Derecha': 30},
            'Bus': {'Izquierda': 5, 'Recto': 20, 'Derecha': 2},
            'Camion': {'Izquierda': 10, 'Recto': 40, 'Derecha': 5},
        },
        'Salidas': {
            'Auto': {'Izquierda': 45, 'Recto': 190, 'Derecha': 35},
            'Bus': {'Izquierda': 4, 'Recto': 18, 'Derecha': 3},
            'Camion': {'Izquierda': 9, 'Recto': 38, 'Derecha': 6},
        },
    },
}

np.random.seed(42)
for node in nodes[1:]:
    traffic_counts[node] = {
        'Entradas': {},
        'Salidas': {},
    }
    for mode in modes:
        traffic_counts[node]['Entradas'][mode] = {
            movement: np.random.randint(20, 250) for movement in movements
        }
        traffic_counts[node]['Salidas'][mode] = {
            movement: np.random.randint(20, 250) for movement in movements
        }

# Paso 3: Balance de Flujos
for node in nodes:
    for mode in modes:
        entradas = traffic_counts[node]['Entradas'][mode]
        salidas = traffic_counts[node]['Salidas'][mode]
        
        total_entradas = sum(entradas.values())
        total_salidas = sum(salidas.values())
        ajuste = total_entradas / total_salidas if total_salidas != 0 else 1
        
        for movement in movements:
            salidas[movement] *= ajuste

# Paso 4: Estimación de la Matriz O-D
OD_matrices = {mode: pd.DataFrame(0, index=nodes, columns=nodes) for mode in modes}

def gravity_model(i, j):
    return 1

for mode in modes:
    for origin in nodes:
        for destination in nodes:
            if origin != destination:
                OD_matrices[mode].loc[origin, destination] = gravity_model(origin, destination)

def ipf(matrix, row_sums, col_sums, max_iter=100, tol=1e-6):
    for _ in range(max_iter):
        matrix = matrix.multiply(row_sums / matrix.sum(axis=1), axis=0)
        matrix = matrix.multiply(col_sums / matrix.sum(axis=0), axis=1)
        if np.allclose(matrix.sum(axis=1), row_sums, atol=tol) and np.allclose(matrix.sum(axis=0), col_sums, atol=tol):
            break
    return matrix

for mode in modes:
    total_origen = pd.Series(0, index=nodes)
    total_destino = pd.Series(0, index=nodes)
    for node in nodes:
        entradas = traffic_counts[node]['Entradas'][mode]
        salidas = traffic_counts[node]['Salidas'][mode]
        total_origen[node] = sum(salidas.values())
        total_destino[node] = sum(entradas.values())
    
    OD_matrices[mode] = ipf(OD_matrices[mode], total_origen, total_destino)

# Resultados
for mode in modes:
    print(f"Matriz O-D para {mode}:")
    print(OD_matrices[mode])
    print("\n")


Matriz O-D para Auto:
           1           2           3           4          5           6
1   0.000000   56.537842   66.582594   69.417681  34.379795   53.082120
2  56.537855    0.000000  112.170377  116.946593  57.918960   89.426396
3  66.582584  112.170334    0.000000  137.723767  68.209061  105.314227
4  69.417635  116.946486  137.723694    0.000000  71.113366  109.798450
5  34.379795   57.918946   68.209071   71.113414   0.000000   54.378808
6  53.082130   89.426392  105.314263  109.798545  54.378818    0.000000


Matriz O-D para Bus:
          1           2           3           4          5           6
1  0.000000    5.475824    5.736726    7.298086   2.914204    5.575175
2  5.475834    0.000000   97.154953  123.597533  49.353814   94.418982
3  5.736738   97.154974    0.000000  129.486511  51.705346   98.917707
4  7.298037  123.596471  129.485370    0.000000  65.777366  125.838945
5  2.914205   49.353740   51.705257   65.777832   0.000000   50.249190
6  5.575186   94.418991  

C:\Users\luisl\AppData\Local\Temp\ipykernel_51840\107514430.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '490.00000000000006' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  total_origen[node] = sum(salidas.values())
C:\Users\luisl\AppData\Local\Temp\ipykernel_51840\107514430.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '369.99999999999994' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  total_origen[node] = sum(salidas.values())
C:\Users\luisl\AppData\Local\Temp\ipykernel_51840\107514430.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '480.00000000000006' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
 

In [2]:
import pandas as pd
import numpy as np

# Paso 1: Definición de la Red con Accesos Externos
internal_nodes = [1, 2, 3, 4, 5, 6]
entrance_nodes = ['E1', 'E2', 'E3', 'E4']
exit_nodes = ['S1', 'S2', 'S3', 'S4']
external_nodes = entrance_nodes + exit_nodes
nodes = internal_nodes + external_nodes

links = []

# Enlaces internos
links.extend([
    (1, 2), (2, 1),
    (2, 3), (3, 2),
    (3, 6), (6, 3),
    (6, 5), (5, 6),
    (5, 4), (4, 5),
    (4, 1), (1, 4),
    (2, 5), (5, 2)
])

# Conexiones con nodos externos
links.extend([
    ('E1', 1), (1, 'S1'),
    ('E2', 2), (2, 'S2'),
    ('E3', 3), (3, 'S3'),
    ('E4', 4), (4, 'S4')
])

# Paso 2: Recolección de Datos
np.random.seed(42)
modes = ['Auto', 'Bus', 'Camion']

traffic_counts = {}
for link in links:
    origin, destination = link
    traffic_counts[(origin, destination)] = {}
    for mode in modes:
        if origin in internal_nodes and destination in internal_nodes:
            traffic_counts[(origin, destination)][mode] = np.random.randint(50, 200)
        else:
            traffic_counts[(origin, destination)][mode] = np.random.randint(100, 300)

# Paso 3: Balance de Flujos
node_flows = {node: {mode: {'in': 0, 'out': 0} for mode in modes} for node in internal_nodes}

for (origin, destination), counts in traffic_counts.items():
    for mode in modes:
        if destination in internal_nodes:
            node_flows[destination][mode]['in'] += counts[mode]
        if origin in internal_nodes:
            node_flows[origin][mode]['out'] += counts[mode]

for node in internal_nodes:
    for mode in modes:
        inflow = node_flows[node][mode]['in']
        outflow = node_flows[node][mode]['out']
        if outflow != 0:
            adjustment_factor = inflow / outflow
            for (origin, destination), counts in traffic_counts.items():
                if origin == node:
                    counts[mode] *= adjustment_factor

# Paso 4: Estimación de la Matriz O-D
OD_pairs = [(o, d) for o in entrance_nodes for d in exit_nodes]

# Definir rutas (simplificado)
routes = {}
for o in entrance_nodes:
    for d in exit_nodes:
        routes[(o, d)] = [o] + internal_nodes + [d]  # Ruta simplificada

# Crear matriz de asignación
link_indices = list(traffic_counts.keys())
assignment_matrix = pd.DataFrame(0, index=link_indices, columns=OD_pairs)

for od_pair in OD_pairs:
    route = routes.get(od_pair)
    if route:
        for i in range(len(route)-1):
            link = (route[i], route[i+1])
            if link in assignment_matrix.index:
                assignment_matrix.loc[link, od_pair] = 1

# Ajuste de la matriz O-D por modos
OD_matrices = {}
for mode in modes:
    observed_flows_mode = np.array([traffic_counts[link][mode] for link in link_indices])
    A = assignment_matrix.values
    OD_vector_mode, residuals, rank, s = np.linalg.lstsq(A, observed_flows_mode, rcond=None)
    OD_matrices[mode] = pd.DataFrame(OD_vector_mode.reshape(len(entrance_nodes), len(exit_nodes)),
                                     index=entrance_nodes, columns=exit_nodes)

# Paso 6: Preparación para PTV Vissim
vissim_routes = []
for od_pair in OD_pairs:
    o, d = od_pair
    route = routes.get(od_pair)
    if route:
        total_volume = sum(OD_matrices[mode].loc[o, d] for mode in modes)
        vissim_routes.append({
            'Origin': o,
            'Destination': d,
            'Nodes': route,
            'Volume': total_volume
        })

# Resultados
for mode in modes:
    print(f"Matriz O-D para {mode}:")
    print(OD_matrices[mode])
    print("\n")



KeyError: "None of [Index(['E1', 1], dtype='object')] are in the [index]"